# Regression

This notebook covers:

1. Simple Regression with Binary Explanatory Variable.
2. Equivalence between Simple Regression and Comparison Measure.
3. Simple Regression with Continuous Explanatory Variable.
4. Regression with Categorical Explanatory Variable.
5. Multiple Regression in General.
6. Omitted Variables Bias.
7. Regression with Interaction Terms in General.
8. Treatment Effect Heterogeneity by Observable Variables

You need to know by now to understand this notebook:

1. Introduction to Jupyter Notebooks.
2. Introduction to R.
3. Conditional Expectations and Summary Statistics.

## Load Data

Just like the last lesson on summary statistics, I am going to use simulated data in this lesson. In particular, certain concepts are easier to demonstrate when the data is synthetic, so that the true Data Generating Process (DGP) is known.

In [1]:
library(tidyverse)
library(stargazer)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.8     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

Warning message:
“package ‘ggplot2’ was built under R version 4.1.3”
Warning message:
“package ‘tidyr’ was built under R version 4.1.2”
Warning message:
“package ‘readr’ was built under R version 4.1.2”
Warning message:
“package ‘dplyr’ was built under R version 4.1.3”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘stargazer’ was built under R version 4.1.2”

Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 




### Generating Correlated Random Variables

The base of synthetic data starts with random variables generated with a particular desired covariance structure. The simplest case uses a multivariate Normal distribution due to its nice joint and marginal distribution properties. There are tools available to generate correlated random variables with other distributions, but those are beyond the scope of this course.

For this lesson, it is not necessary for variables to have specific meanings. I will assign the variables names corresponding to letters we use when teaching the theory in the course.

In [2]:
set.seed(998)
sim.data = matrix(rnorm(25000, 0, 1), 5000, 5, FALSE)
sim.data <- sim.data %*% chol(matrix(
    c(1, 0, .5, .25, 0,
     0, 1, .6, .1, 0,
     .5, .6, 1, .4, 0,
     .25, .1, .4, 1, 0,
     0, 0, 0, 0, 1),
    5, 5, TRUE))
colnames(sim.data) <- c('D', 'X', 'W', 'Z', 'e')
sim.data <- as_tibble(sim.data)

### Modifying the Random Variables to Desired Types

For our purposes here, we require one binary variable, one categorical variable with more than $2$ categories, and one continuous variable. To adhere as closely to course terminology as possible, `D` will be the binary variable, `W` the categorical variable, and `X` and `Z` continuous variables. 

In [3]:
sim.data <- sim.data %>%
    mutate(
        D = if_else(D > 0, 1, 0),
        W = as_factor(ntile(W, 5))
    )

### Generating the Outcome Variable

I manually define the outcome variable so that I know the true parameters of the CEF of the outcome variable. Keep this in mind as we look at regression outputs later. Notice that despite generating $4$ variables I use only $3$ variables to define the CEF. This is not a mistake and in a future lesson the purpose of `Z` will be made clear.

In [4]:
sim.data <- sim.data %>%
    mutate(
        Y = .25 + 3 * D + .5 * X + .4 * (W == 2) + .7 * (W == 3) + 1.3 * (W == 4) + 1.9 * (W == 5) + e
    )

### Summary Statistics

I report a summary of the data here. I could also use `summarize` or `stargazer`, but this is sufficient for my purposes since this notebook is about teaching how linear regression works in practice and not how to execute a full data analysis pipeline.

In [5]:
summary(sim.data)

       D                X             W              Z            
 Min.   :0.0000   Min.   :-3.688439   1:1000   Min.   :-3.703643  
 1st Qu.:0.0000   1st Qu.:-0.716236   2:1000   1st Qu.:-0.686461  
 Median :1.0000   Median : 0.009939   3:1000   Median : 0.009768  
 Mean   :0.5054   Mean   :-0.007287   4:1000   Mean   :-0.003324  
 3rd Qu.:1.0000   3rd Qu.: 0.685444   5:1000   3rd Qu.: 0.681739  
 Max.   :1.0000   Max.   : 3.675470            Max.   : 3.542278  
       e                   Y          
 Min.   :-3.517587   Min.   :-3.4588  
 1st Qu.:-0.690418   1st Qu.: 0.7849  
 Median : 0.006281   Median : 2.5836  
 Mean   :-0.012897   Mean   : 2.6097  
 3rd Qu.: 0.671192   3rd Qu.: 4.4280  
 Max.   : 3.381951   Max.   : 8.6636  

## Simple Regression with Binary Explanatory Variable

A simple regression is a linear regression model with one explanatory variable. Formally, it is written as

$$
    Y_{i} = \alpha + \beta D_{i} + e_{i}.
$$

$Y_{i}$ is our outcome variable, $D_{i}$ is the explanatory variable and $e_{i}$ is the unexplained residual term. Under the mean independence assumption

$$
    \mathbb{E}[e_{i} \mid D_{i}] = 0,
$$

the simple linear regression model implies the CEF

$$
    \mathbb{E}[Y_{i} \mid D_{i}] = \alpha + \beta D_{i}.
$$

When $D_{i}$ is a discrete variable---specifically a binary variable---the parameters $\alpha$ and $\beta$ have the following definitions:

$$
    \alpha = \mathbb{E}[Y_{i} \mid D_{i} = 0]
$$

and

$$
    \beta = \mathbb{E}[Y_{i} \mid D_{i} = 1] - \mathbb{E}[Y_{i} \mid D_{i} = 0].
$$

These definitions provide the blueprint for interpreting these parameters. $\alpha$ is the expected outcome if the explanatory variable has a value of $0$. $\beta$ is the change in the expected outcome when the explanatory variable changes value from $0$ to $1$.

For example, if $Y_{i}$ is blood pressure and $D_{i}$ is the state of employed ($D_{i}=1$) or unemployed ($D_{i}=0$), then $\alpha$ estimates the expected blood pressure level of unemployed people and $\beta$ estimates how much greater or less the expected blood pressure level of employed people are compared to unemployed people.

In [6]:
sim.data %>%
    lm(Y ~ D, data = .) %>%
    stargazer(., type = 'text')


                        Dependent variable:    
                    ---------------------------
                                 Y             
-----------------------------------------------
D                            3.525***          
                              (0.040)          
                                               
Constant                     0.828***          
                              (0.029)          
                                               
-----------------------------------------------
Observations                   5,000           
R2                             0.604           
Adjusted R2                    0.604           
Residual Std. Error      1.428 (df = 4998)     
F Statistic         7,619.839*** (df = 1; 4998)
Note:               *p<0.1; **p<0.05; ***p<0.01


### Interpretation

As `D` is a binary variable,

1. The `Constant` implies that the mean of outcome `Y` for all individuals with `D = 0` is estimated to be $0.828$.
2. The coefficient on `D` implies that the mean outcome `Y` for all individuals with `D = 1` is estimated to be $0.828 + 3.525 = 4.353$. The outcome of the `D = 1` group is greater than the outcome of the `D = 0` group.

## Manually Estimating the Simple Linear Regression Model

We can also test the theory described in the lecture notes by manually estimating our simple linear regression model and comparing the estimates to the output from `lm`.

### Point Estimate of the Effect of the Explanatory Variable

First, the point estimate on the explanatory variable is given by

$$
    \widehat{\beta} = \frac{\widehat{Cov}(D_{i},Y_{i})}{\widehat{Var}(D_{i})}.
$$

In [7]:
cov(sim.data$D, sim.data$Y) / var(sim.data$D)

[1] 3.524784

### Point Estimate of the Constant

Next, the point estimate on the constant is given by

$$
    \widehat{\alpha} = \frac{1}{N} \sum_{i=1}^{N} \left( Y_{i} - \widehat{\beta} D_{i} \right).
$$

In [8]:
mean(
    sim.data$Y - sim.data$D * (cov(sim.data$D, sim.data$Y) / var(sim.data$D))
)

[1] 0.8282331

### Standard Error of the Effect of the Explanatory Variable

We move on to the uncertainty of the estimates. Before we begin, it is important to define and remember the estimated residual term

$$
    \widehat{e}_{i} = Y_{i} - (\widehat{\alpha} + \widehat{\beta} D_{i}) = Y_{i} - \widehat{Y}_{i}.
$$

Then, the variance of the point estimate on the explanatory variable is given by

$$
    \widehat{Var}(\widehat{\beta}) = \frac{\widehat{Var}(\widehat{e}_{i})}{\widehat{Var}(D_{i})}
$$

and the SE is given by

$$
    \widehat{SE}(\widehat{\beta}) = \sqrt{\frac{\widehat{Var}(\widehat{\beta})}{N}}.
$$

In [9]:
sqrt(
    var(resid(lm(Y ~ D, data = sim.data))) / var(sim.data$D) / nrow(sim.data)
)

[1] 0.0403713

### Standard Error of the Constant

Finally, the standard error of the constant is given by

$$
    \widehat{SE}(\widehat{\alpha}) = \widehat{SE}(\widehat{\beta}) \sqrt{\frac{1}{N} \sum_{i=1}^{N} D_{i}^{2}}
$$

which in the case of a binary variable is equal to

$$
    \widehat{SE}(\widehat{\alpha}) = \widehat{SE}(\widehat{\beta}) \sqrt{\overline{D}}
$$


In [10]:
sqrt(
    (var(resid(lm(Y ~ D, data = sim.data))) / var(sim.data$D)) * mean(sim.data$D ^ 2) / nrow(sim.data)
)

[1] 0.02870056

### Computing $R^{2}$

$R^{2}$ is the Explained Sum of Squares (ESS) of the model. It tells us how much of the variation in our outcome variable is explained by the simple linear regression model. Mathematically, the Residual Sum of Squares (RSS) is the ratio of the variance of the estimated residuals to the variance of the outcome variable and the ESS is just $1$ minus the RSS;

$$
    R^{2} = 1 - \frac{\widehat{Var}(\widehat{e}_{i})}{\widehat{Var}(Y_{i})}
$$

In [11]:
1 - var(resid(lm(Y ~ D, data = sim.data))) / var(sim.data$Y)

[1] 0.6038941

### Interpreting $R^{2}$

What this number means is that with just a binary explanatory variable, our simple linear regression model explains $60.4\%$ of the total variation in our outcome variable. Unlike in this synthetic data set and unlike in predictive data science applications, the $R^{2}$ of most applied econometric models on real world data is not so significant in general.

## Comparing this to the Comparison Measure

You may have noticed that the definition of $\beta$ in a simple linear regression model with a binary explanatory variable is similar to the comparison measure. That is because it is! These two models describe the same parameter. You can see it by comparing the output from the $2$-sample $t$-test with the equal group variance assumption to the output from the simple linear regression model. First I load `broom` so that I can display $t$-test results in a neat table format instead.

In [12]:
library(broom)

Warning message:
“package ‘broom’ was built under R version 4.1.3”


### Running the $t$-test

In [13]:
map_df(list(
    t.test(
        sim.data$Y[sim.data$D==1],
        sim.data$Y[sim.data$D==0],
        var.equal = TRUE
    )
), tidy)

estimate,estimate1,estimate2,statistic,p.value,parameter,conf.low,conf.high,method,alternative
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
3.524784,4.353017,0.8282331,87.29169,0,4998,3.445623,3.603946,Two Sample t-test,two.sided


### Checking Against the Simple Linear Regression Model

Notice that the estimate is exactly the estimated coefficient on the binary explanatory variable in our simple linear regression model, estimate $2$ is the estimated constant, and estimate $1$ is what I said is the total outcome of the `D = 1` group. As a final check, I estimate the $95\%$ confidence interval of the simple linear regression model to confirm that the confidence interval of the difference in the $2$-sample $t$-test is the same as the confidence interval of the simple linear regression model.

In [14]:
confint(
    lm(Y ~ D, data = sim.data),
    level = .95
)

,2.5 %,97.5 %
(Intercept),0.7719561,0.884510
D,3.4456229,3.603946


## Simple Regression with Continuous Explanatory Variable

The theory of the simple linear regression model with a continuous explanatory variable is not much more complicated. First, the model is estimated the same way, both when seeking to do so manually and when using `lm` in `R`. Second, only the interpretation of the coefficient on the explanatory variable changes. Notice that under the implied CEF

$$
    \mathbb{E}[Y_{i} \mid X_{i}] = \alpha + \beta X_{i}
$$

the constant still has the interpretation

$$
    \alpha = \mathbb{E}[Y_{i} \mid X_{i} = 0].
$$

Whether this interpretation makes sense depends on your data. For example, if your regression is income versus age, then anything other than a zero is not realistic. For this reason, in many econometric applications the constant is neither reported nor discussed. This also points to a practical implication of the linear regression model: if the interpretation of the constant in your model does not make sense, the most likely explanation is that the linear regression model is only an approximation of the true Data Generating Process of your data.

However, now the coefficient on the explanatory variable has the interpretation

$$
    \beta = \frac{d \mathbb{E}[Y_{i} \mid X_{i}]}{d X_{i}}.
$$

It is the marginal change in the outcome given a marginal increase in the explanatory variable. It is also alternatively interpretable as

$$
    \beta = \mathbb{E}[Y_{i} \mid X_{i} = x + 1] - \mathbb{E}[Y_{i} \mid X_{i} = x]
$$

for any arbitrary $x$ in the support of $X$. This tells us that a unit increase in the explanatory variable $X_{i}$ will be associated with a $\beta$ change in the outcome $Y_{i}$ on average. This interpretation especially makes sense if your explanatory variable is discrete but cardinal. For example, a weight variable that is reported rounding to the nearest $0.1kg$ is a discrete variable but its values have cardinal meaning because we know exactly what it means for someone to have a weight $1kg$ greater than someone else.

In [15]:
sim.data %>%
    lm(Y ~ X, data = .) %>%
    stargazer(., type = 'text')


                        Dependent variable:    
                    ---------------------------
                                 Y             
-----------------------------------------------
X                            0.835***          
                              (0.029)          
                                               
Constant                     2.616***          
                              (0.030)          
                                               
-----------------------------------------------
Observations                   5,000           
R2                             0.144           
Adjusted R2                    0.144           
Residual Std. Error      2.098 (df = 4998)     
F Statistic          844.183*** (df = 1; 4998) 
Note:               *p<0.1; **p<0.05; ***p<0.01


### Interpreting the Regression Coefficients

These estimation results imply that on average, a unit increase in `X` is associated with a $0.835$ unit increase in `Y`. What this means in practice depends on what `Y` and `X` are. There are four main situations of interest.

1. Your `Y` and `X` are in levels. For example, you regressed the weight of newborns measured in kg on the estimated gestation period measured in weeks. Then $\beta$ has the implication that an additional week of gestation is associated with a $\beta$ increase in the weight of the newborn.
2. Your `Y` is in levels and your `X` is in logs. What this means is that you are running the regression
$$
    Y_{i} = \alpha + \beta \underbrace{\ln(Z_{i})}_{X_{i}} + e_{i}.
$$
In other words, `X` is the log-transform of some other variable `Z` and then you regress `Y` on `X`. Because $d \ln(Z) = d Z / Z$, $\beta$ has the implication that a percent increase in `Z` is associated with a $\beta$ unit increase in `Y`. This is also known as a partial elasticity. For example, if you regressed the number of years of schooling attained by a child on the log of the average annual total earnings of their parents, $\beta$ has the implication that increasing the annual total earnings of the child's parent's total earnings by one percent is associated with a $\beta$ increase in the number of years of schooling attained by the child.
3. Your `Y` is in logs and your `X` is in levels. In other words, your regression is
$$
    \underbrace{\ln(Z_{i})}_{Y_{i}} = \alpha + \beta X_{i} + e_{i}.
$$
By similar logic, a unit increase in `X` is associated with a $\beta$ percent increase in `Z`. For example, if you regressed the log wealth at $65$ on the years of schooling attained, then $\beta$ has the implication that an additional year of schooling is associated with a $\beta \times 100\%$ increase in the log wealth at $65$. 
4. Both `Y` and `X` are in logs. In other words, your regression is
$$
    \underbrace{\ln(Z_{i})}_{Y_{i}} = \alpha + \beta \underbrace{\ln(W_{i})}_{X_{i}} + e_{i}.
$$
Here, a percent increase in `X` is associated with a $\beta$ percent increase in `Z`. In other words, $\beta$ here estimates an elasticity. For example, if you regressed the log annual revenue of a firm on the log amount spent on advertising, then $\beta$ has the implication of being the revenue elasticity of advertising. A percent increase in advertising spending is associated with a $\beta \times 100\%$ increase in revenue.

### Checking Against Manual Calculations

We can check that a manual computation of the coefficient on a continuous explanatory variable matches the result that `lm` produces. I will not show the rest as it should be clear that `lm` does indeed follow the formulas that were taught in class, but you are welcome to check them yourself. 

In [16]:
cov(sim.data$X, sim.data$Y) / var(sim.data$X)

[1] 0.8349092

## Exercise Set 1: Regression with Binary Outcome Variable

Do you really understand how to run a simple regression and interpret its output? This exercise will test your understanding. To begin, I generate a data set with a twist. You will see in due time in this course what model this attempts to approximate. 

In [17]:
ex.data = matrix(rnorm(25000, 0, 1), 5000, 5, FALSE)
ex.data <- ex.data %*% chol(matrix(
    c(1, .5, .8, 0, 0,
     .5, 1, .25, 0, 0,
     .8, .25, 1, 0, 0,
     0, 0, 0, 1, .5,
     0, 0, 0, .5, 1),
    5, 5, TRUE))
colnames(ex.data) <- c('X', 'W', 'Z', 'u', 'e')
ex.data <- as_tibble(ex.data)

In [18]:
ex.data <- ex.data %>%
    mutate(
        D = if_else(.25 - 1.8 * Z + u > 0, 1, 0),
        W = as_factor(ntile(W, 5))
    ) %>%
    mutate(
        Y = .5 + 1.5 * D + X + .5 * D * X + .4 * (W == 2) + .7 * (W == 3) + 1.2 * (W == 4) + 2. * (W == 5) + e
    )

### Part 1: Run the regression

Suppose you want to regress the binary variable `D` on the explanatory variable `Z`. How will you do it?

### Part 2: Interpret the Output

Great! Now, what do the numbers in the table mean? To help you out, what I mean is the following: What is the probability of `D = 1` when `Z = 0`? And what is the probability of `D = 1` on average in the entire data set?

Hint for the second question: The marginal distribution of `Z` is Normal with mean zero.

### Part 3: Taking Stock of the Results

Now that you know how to interpret the regression results for a simple linear regression with a binary outcome variable, there is one last interesting question for you to ask yourself: Do the results make sense?

To help you out, what I mean is the following: What are the limits of the range of values for `Z` for which these results make sense? As a hint, go back to Part 2 and ask whether there is a point for `Z` for which that interpretation no longer makes sense.

## Regression with Categorical Variable

Categorical variables are interesting, because for most part their category labels have at most ordinal value, meaning that it does not make sense in general to directly run the regression

$$
    Y_{i} = \alpha + \beta W_{i} + e_{i}
$$

because in general

$$
    \beta = \mathbb{E}[Y_{i} \mid W_{i} = w + 1] - \mathbb{E}[Y_{i} \mid W_{i} = w]
$$

is not interpretable. Instead, what we do in general is to split up categorical variables into dummy variables for each category (level) and run a multiple regression with all the dummies instead,

$$
    Y_{i} = \alpha + \sum_{k=1}^{K-1} \beta_{k} \mathbf{1}\{W_{i} = k\} + e_{i}.
$$

Here, $\mathbf{1}$ represents the indicator variable and we specifically sum only over $K-1$ categories, leaving one group out. The reason for leaving one out is better explained in terms what what happens if we do not do this and instead run

$$
    Y_{i} = \alpha + \sum_{k=1}^{K} \beta_{k} \mathbf{1}\{W_{i} = k\} + e_{i}.
$$

Then,

$$
    \mathbb{E}[Y_{i} | W_{i} = k] = \alpha + \beta_{k}
$$

for all $k$. The issue is that $\alpha$ is a common term to all $K$ levels and hence there are infinite possible solutions because

$$
    \mathbb{E}[Y_{i} | W_{i} = k] = (\alpha - \gamma) + (\beta_{k} + \gamma)
$$

for any $\gamma$ is also a valid solution. This problem is known as collinearity and under the hood the problem is that this flawed regression model defines a design matrix that does not have full rank and therefore cannot be inverted. The solution is to break this collinearity by dropping one of the levels.

In `R`, first of all `factor` variables are automatically treated this way. This is why it is important to always set variables as `factor` if they are indeed categorical variables. Second, `R` also automatically takes care of the leave-one-out requirement for you, unless you explicitly ask `R` to drop the constant instead. In practical terms:

In [19]:
stargazer(
    sim.data %>%
        mutate(W = relevel(W, 5)) %>%
        lm(Y ~ W, data = .),
    sim.data %>%
        lm(Y ~ W, data = .),
    sim.data %>%
        lm(Y ~ W - 1, data = .)
    , type = 'text')


                                                                Dependent variable:                                
                                -----------------------------------------------------------------------------------
                                                                         Y                                         
                                            (1)                         (2)                         (3)            
-------------------------------------------------------------------------------------------------------------------
W1                                       -4.449***                                               0.461***          
                                          (0.075)                                                 (0.053)          
                                                                                                                   
W2                                       -3.306***                   1.

### Interpreting the Regression Coefficients

Notice that in the first two models with the constant, one of the levels was dropped automatically without having to explicitly state it to `R`. The first column drops `W = 5` because we asked `R` to treat `W = 5` as the new reference level using the `relevel` function. The second column drops `W = 1` because by default `R` always sets the first level in alphanumeric sort order as the reference level. Notice that the constant in both cases corresponds to the estimate for that omitted level in the third column without the constant and all the coefficient estimates are the difference between the constant in that model that the corresponding coefficient for that level in the third column. This therefore gives us our interpretations:

1. When your model contains a constant, i.e.
$$
    Y_{i} = \alpha + \sum_{k=1}^{K-1} \beta_{k} \mathbf{1}\{W_{i} = k\} + e_{i},
$$
then
$$
    \alpha = \mathbb{E}[Y_{i} \mid W_{i} = K]
$$
and
$$
    \beta_{k} = \mathbb{E}[Y_{i} \mid W_{i} = k] - \mathbb{E}[Y_{i} \mid W_{i} = K].
$$
In other words, $\alpha$ is simply the mean outcome of the left-out group and each $\beta$ estimates the difference in the mean outcome between the left-out group and the target group.
2. When your model does not contain a constant, i.e.
$$
    Y_{i} = \sum_{k=1}^{K} \beta_{k} \mathbf{1}\{W_{i} = k\} + e_{i},
$$
then
$$
    \beta_{k} = \mathbb{E}[Y_{i} \mid W_{i} = k].
$$
In other words, $\beta$ directly estimates the mean outcome of the target group.

Therefore, in this example the mean outcome of the `W = 1` group is $0.461$ and then the `W = 2` group has a $1.143$ greater outcome than the `W = 1` group on average, resulting in a total outcome of $1.604$ for the `W = 2` group on average.

## Multiple Regression in General

Regression with categorical variables is a special case of multiple regression. In general, we can always regress on multiple variables

$$
    Y_{i} = \alpha + \beta_{1} X_{1,i} + \beta_{2} X_{2,i} + \cdots + \beta_{K} X_{K,i} + e_{i}.
$$

Here, the coefficient interpretations change slightly because

$$
    \alpha = \mathbb{E}[Y_{i} \mid X_{1,i} = 0, X_{2,i} = 0, \cdots, X_{K,1} = 0]
$$
and
$$
    \beta_{j} = \frac{\partial \mathbb{E}[Y_{i} \mid \{X_{k,i}\}_{k=1,2,\cdots,K}]}{\partial X_{j,i}}.
$$

In words, $\alpha$ is the expected outcome when all explanatory variables are zero and $\beta_{j}$ is the estimated association between $X_{j,i}$ and $Y_{i}$ holding all other explanatory variables constant. i.e., a unit increase in $X_{j,i}$ holding constant all other explanatory variables is associated with a $\beta_{j}$ change in $Y_{i}$. Again, whether $\alpha$ makes sense to interpret depends on your model, and $\beta$s can take on either the proportional association, partial elasticity, or elasticity interpretations depending on whether your outcome and explanatory variables are measured in logs or levels.

In `R`, you simply expand the model in `lm` with more variables.

In [20]:
sim.data %>%
    lm(Y ~ D + X, data = .) %>%
    stargazer(., type = 'text')


                        Dependent variable:    
                    ---------------------------
                                 Y             
-----------------------------------------------
D                            3.544***          
                              (0.032)          
                                               
X                            0.854***          
                              (0.015)          
                                               
Constant                     0.825***          
                              (0.023)          
                                               
-----------------------------------------------
Observations                   5,000           
R2                             0.755           
Adjusted R2                    0.755           
Residual Std. Error      1.123 (df = 4997)     
F Statistic         7,702.292*** (df = 2; 4997)
Note:               *p<0.1; **p<0.05; ***p<0.01


### Interpreting the Regression Coefficients

Here, these estimates suggest that

1. Holding `X` constant, the `D = 1` group has a $3.544$ higher outcome on average than the `D = 0` group.
2. Holding `D` constant, a unit increase in `X` is associated with a $0.854$ increase in the outcome `Y`.

## Exercise Set 2: Multiple Regression

Using the same exercise data set as before, it is now your turn to try multiple regression.

### Part 1: Run the Regression

Suppose we believe `Y` is simply a linear function of `X`, `D` and `W`. How do you run the regression?

### Part 2: Interpret the Results

Let me be specific here. Try out these three things:

1. What is the expected outcome value of having `D = 1`, `W = 1`, and `X = 1`?
2. What is the expected outcome value of having `D = 0`, `W = 5`, and `X = 0`?
3. What is the expected difference in outcome between having `D = 1` and `W = 5` and `D = 0` and `W = 1`?

### Part 3: See Something Wrong?

Look at the coefficients carefully again. Is something wrong with the coefficients? Which assumption do you think we have violated here?

## Omitted Variables Bias

If you have been paying attention to the magnitudes of our estimates and recall the original true model that we used to define our outcome variable, it should be clear that the coefficients in the simple linear regression models are wildly misestiamted. This is a live example of omitted variables bias. Compare the results of the three simple models below against the fully, correctly-specified model.

In [21]:
stargazer(
    sim.data %>%
        lm(Y ~ D, data = .),
    sim.data %>%
        lm(Y ~ X, data = .),
    sim.data %>%
        lm(Y ~ W, data = .),
    sim.data %>%
        lm(Y ~ D + X + W, data = .),
    type = 'text', df = FALSE)


                                   Dependent variable:               
                    -------------------------------------------------
                                            Y                        
                        (1)         (2)         (3)          (4)     
---------------------------------------------------------------------
D                     3.525***                             3.026***  
                      (0.040)                              (0.032)   
                                                                     
X                                 0.835***                 0.478***  
                                  (0.029)                  (0.018)   
                                                                     
W2                                            1.143***     0.404***  
                                              (0.075)      (0.047)   
                                                                     
W3                 

## Verifying the OVB Equation

We mentioned before that when

$$
    Y_{i} = \alpha + \beta X_{i} + \gamma W_{i} + \epsilon_{i}
$$

is misspecified as

$$
    Y_{i} = a + b X_{i} + e_{i},
$$

the misspecified coefficient takes on the value

$$
    b = \beta + \gamma \frac{Cov(X_{i},W_{i})}{Var(X_{i})}.
$$

You will recognise the fraction term in the OVB to be basically the coefficient of a simple linear regression of $X_{i}$ on $W_{i}$. This gives us a way to confirm how OVB works in our simulated data.

1. First, we estimate the two misspecified models `D ~ X + W` and `Y ~ X + W` and the true model `Y ~ X + D + W`.
2. Then, we can compare whether the coefficient on `X` in the misspecified model `Y ~ X + W` is really the true coefficient on `X` in `Y ~ X + D + W` plus the coefficient on `D` multiplied by the coefficient on `X` in `D ~ X + W`.

In [22]:
stargazer(
    sim.data %>%
        lm(Y ~ X + D + W, data = .),
    sim.data %>%
        lm(Y ~ X + W, data = .),
    sim.data %>%
        lm(D ~ X + W, data = .),
    type = 'text', df = FALSE)


                           Dependent variable:        
                    ----------------------------------
                               Y                D     
                        (1)         (2)        (3)    
------------------------------------------------------
X                     0.478***     -0.022   -0.165*** 
                      (0.018)     (0.028)    (0.007)  
                                                      
D                     3.026***                        
                      (0.032)                         
                                                      
W2                    0.404***    1.155***   0.248*** 
                      (0.047)     (0.076)    (0.020)  
                                                      
W3                    0.803***    2.121***   0.436*** 
                      (0.050)     (0.079)    (0.021)  
                                                      
W4                    1.328***    3.076***   0.578*** 
         

In [23]:
print(
    list('OVB' = 0.478 + 3.026 * (-0.165))
)

$OVB
[1] -0.02129



### Final Remarks about OVB

1. See that the manually-computed biased coefficient basically matches the actual biased estimate in the misspecified model.
2. Notice that although the partial effect of `X` is in fact positive, due to the direction of the bias the biased estimate is statistically indistinguishable from zero. OVB can obscure your desired treatment effect estimate in unexpected ways, which is why as an analyst you will still be required to bring your domain knowledge to the analysis to determine the likely direction of the bias when you do not know the true Data Generating Process as in this case.

## Regression with Interaction Terms

We are not done yet. With multiple regression, we can now also discuss regression models containing interaction terms. For example with two variables,

$$
    Y_{i} = \alpha + \beta X_{i} + \gamma D_{i} + \delta X_{i} \times D_{i} + e_{i}.
$$

This is a general procedure because suppose you just want a quadratic model, it is just like interacting the explanatory variable with itself,

$$
    Y_{i} = \alpha + \beta X_{i} + \gamma X_{i} \times X_{i} + e_{i}.
$$

In `R`, you estimate a model with interaction terms using the `*` operator.

In [24]:
stargazer(
    sim.data %>%
        lm(Y ~ X * D, data = .),
    type = 'text', df = FALSE)


                        Dependent variable:    
                    ---------------------------
                                 Y             
-----------------------------------------------
X                            0.835***          
                              (0.022)          
                                               
D                            3.545***          
                              (0.032)          
                                               
X:D                            0.038           
                              (0.031)          
                                               
Constant                     0.825***          
                              (0.023)          
                                               
-----------------------------------------------
Observations                   5,000           
R2                             0.755           
Adjusted R2                    0.755           
Residual Std. Error            1.123   

### Interpreting the Regression Coefficients

With interactions, our interpretations change quite significantly. Consider for $X_{i}$,

$$
    \frac{\partial \mathbb{E}[Y_{i} \mid X_{i}, D_{i}]}{\partial X_{i}} = \beta + \delta D_{i}.
$$

This means that the partial effect of an explanatory variable is not constant when it interacts with another variable. In our case above, the estimates suggest that the partial effect of `X` on `Y` is $0.835$ if `D = 0` and $0.835 + 0.038 = 0.872$ if `D = 1`.

Of course, we know that the true model has no such interaction (we generated the synthetic data ourselves) and this shows up in the fact that the coefficient on the interaction term is statistically indistinguishable from zero. This has the distinct implication that suppose you are interested in treatment effect heterogeneity---whether the average effect of some treatment variable $D_{i}$ depends on other observable characteristics $X_{i}$. Then, you simply test whether such heterogeneity exists by estimating a model with interaction terms between your treatment and observable characteristics variables.

## Exercise Set 3: Regression with Interaction Term

Now we add interaction terms to our regression models.

### Part 1: Running the Regression

Now, run what seems to be the true CEF for `Y`.

### Part 2: Interpreting the Estimates

Specifically, what is the difference in expected outcome between `D = 1` and `D = 0` when `W = 1` and `X = 0`? What about when `X = 2`? What about instead when `W = 5`?

### Part 3: See Something Wrong?

Compare the regression estimates to the true parameters of `Y`. What seems to be wrong? What assumption have we violated?

You will learn how to deal with this problem later in the course.

## OVB when Interaction is Present

To cap things off, I showcase what happens with OVB when your true model has an interaction. To begin, I define a new outcome variable that does include treatment effect heterogeneity. Then, I will estimate several misspecified models and the true models and discuss what the OVB looks like.

In [25]:
sim.data <- sim.data %>%
    mutate(
        A = X + 2.5
    ) %>%
    mutate(
        K = 1 + 2 * D + 4 * A + 1.5 * D * A + e
    )

In [26]:
stargazer(
    sim.data %>%
        lm(K ~ D, data = .),
    sim.data %>%
        lm(K ~ A, data = .),
    sim.data %>%
        lm(K ~ A + D, data = .),
    sim.data %>%
        lm(K ~ A * D, data = .),
    type = 'text', df = FALSE)


                                     Dependent variable:                  
                    ------------------------------------------------------
                                              K                           
                        (1)           (2)           (3)           (4)     
--------------------------------------------------------------------------
D                     5.664***                   5.772***      2.028***   
                      (0.143)                     (0.036)       (0.075)   
                                                                          
A:D                                                            1.502***   
                                                                (0.028)   
                                                                          
A                                  4.698***      4.729***      3.982***   
                                    (0.043)       (0.017)       (0.020)   
                        

### Manual OVB Computation

To skip the mathematics, the OVB when the true model

$$
    Y_{i} = \alpha + \beta X_{i} + \gamma D_{i} + \delta X_{i} \times D_{i} + \epsilon_{i}
$$

is misspecified as

$$
    Y_{i} = a + b X_{i} + g D_{i} + e_{i}
$$

is

$$
    g = \gamma + \delta \mathbb{E}[X_{i}].
$$

For our synthetic data, see that $\gamma = 2$, $\delta = 1.5$, and recall that because we are using $X_{i}$ shifted forward by $2.5$ when the original $\mathbb{E}[X_{i}] = 0$, for this exercise we have $\mathbb{E}[X_{i}] = 2.5$. We can now do the computation:

In [27]:
print(
    list('OVB Theory' = 2 + 1.5 * 2.5, 'OVB Estimate' = 2.028 + 1.502 * mean(sim.data$A))
)

$`OVB Theory`
[1] 5.75

$`OVB Estimate`
[1] 5.772054



### Final Notes about OVB

These numbers basically match the biased estimate in the third column. You can do even more math to figure out how the biased estimators in the first two columns are computed but it is for our purposes not necessary. The principal message to take away here is that even when you do observe all relevant variables, mispecifying the relationship between the variables can also cause omitted variables bias. This does not have to come from only interaction terms. Including variables using the wrong functional forms can also cause bias, e.g. using linear instead of quadratic or levels instead of logs.

## Exercise Set 4: Running Regressions for your Project

The rest of this exercise guides you through doing regressions for your Project.

### Part 1: What are your Variables?

First, load your data and describe what variables you intend to use.

In [28]:
library(haven)

Warning message:
“package ‘haven’ was built under R version 4.1.3”


In [29]:
prj.data <- 'load your data with the appropriate command'

In [30]:
prj.data <- 'do any necessary data cleaning and processing'

In [31]:
stargazer(
    data.frame(prj.data),
    type = 'text'
)


Statistic N Mean St. Dev. Min Max


### Part 2: Run a Simple Regression

Always start simple and see what the simple linear relationship between your outcome and treatment variable is.

In [32]:
reg.mod.1 <- 'fill in the lm command'
stargazer(
    reg.mod.1,
    type = 'text'
)


fill in the lm command
----------------------


### Part 3: Gradually Increase the Complexity

Now that we have an established base fact, start adding control variables that you think may be useful.

In [33]:
reg.mod.2 <- 'add a control to your model'
reg.mod.3 <- 'add another control to your model'
reg.mod.4 <- 'add yet another control to your model'
reg.mod.5 <- 'continue creating models with more controls as needed and remember to add them to stargazer to compare model estimates!'
stargazer(
    reg.mod.1,
    reg.mod.2,
    reg.mod.3,
    type = 'text'
)


fill in the lm command
----------------------

add a control to your model
---------------------------

add another control to your model
---------------------------------


### Part 4: Interpret Your Results

Do a write-up interpreting your results. If you are lost on what to discuss, here are some tips:

1. What do the plain coefficients actually mean about the relationship between your outcome and control variables?
2. Which coefficients are statistically significant? What does this mean?
3. Does your treatment variable coefficient change as you add more controls? What does this mean?
4. Taken as a whole, what story do these estimates tell us about your outcome variable?

### Part 5: Export Your Results

The text format output of `stargazer` is great for viewing in a Notebook but less so when writing academic reports of the standard expected of Economics undergraduates. Instead, and since most of you have expressed that you do not know `LaTeX` and are not especially keen on learning, we do the following:

1. Have `stargazer` export the results in HTML format.
2. Open the HTML file in your web browser.
3. Copy the table into your Word document---Word automatically converts HTML tables into Word format tables.

In [34]:
stargazer(
    'add all regression results that you want to export',
    type = 'html',
    out = 'prj_ols_reg_out.html',
    out.header = TRUE
)


<table style="text-align:center"><tr><td colspan="1" style="border-bottom: 1px solid black"></td></tr><tr><td>add all regression results that you want to export</td></tr>
<tr><td colspan="1" style="border-bottom: 1px solid black"></td></tr></table>
